# 영업 성공 여부 분류 경진대회

# 할 것

결측치 이상치 처리
파생변수 생성

## 1. 데이터 확인

### 필수 라이브러리

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
)
from sklearn.model_selection import StratifiedKFold
from sklearn.utils.class_weight import compute_class_weight
from catboost import CatBoostClassifier
from sklearn.metrics import f1_score

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

def seed_everything(seed):
    np.random.seed(seed) #numpy를 사용할 경우 고정
seed_everything(42)

### 데이터 셋 읽어오기

In [3]:
df_train = pd.read_csv("train.csv") # 학습용 데이터
df_test = pd.read_csv("submission.csv") # 테스트 데이터(제출파일의 데이터)

In [5]:
df_train[:10]

,bant_submit,customer_country,business_unit,com_reg_ver_win_rate,customer_idx,customer_type,enterprise,historical_existing_cnt,id_strategic_ver,it_strategic_ver,idit_strategic_ver,customer_job,lead_desc_length,inquiry_type,product_category,product_subcategory,product_modelname,customer_country.1,customer_position,response_corporate,expected_timeline,ver_cus,ver_pro,ver_win_rate_x,ver_win_ratio_per_bu,business_area,business_subarea,lead_owner,is_converted
0,1.00,/Quezon City/Philippines,AS,0.066667,32160,End-Customer,Enterprise,NaN,NaN,NaN,NaN,purchasing,62,Quotation or purchase consultation,multi-split,NaN,NaN,/Quezon City/Philippines,entry level,LGEPH,less than 3 months,1,0,0.003079,0.026846,corporate / office,Engineering,0,True
1,1.00,/PH-00/Philippines,AS,0.066667,23122,End-Customer,Enterprise,12.0,NaN,NaN,NaN,media and communication,96,Quotation or purchase consultation,multi-split,NaN,NaN,/PH-00/Philippines,ceo/founder,LGEPH,less than 3 months,1,0,0.003079,0.026846,corporate / office,Advertising,1,True
2,1.00,/Kolkata /India,AS,0.088889,1755,End-Customer,Enterprise,144.0,NaN,NaN,NaN,engineering,56,Product Information,single-split,NaN,NaN,/Kolkata /India,partner,LGEIL,less than 3 months,1,0,0.003079,0.026846,corporate / office,Construction,2,True
3,1.00,/Bhubaneswar/India,AS,0.088889,4919,End-Customer,Enterprise,NaN,NaN,NaN,NaN,entrepreneurship,44,Quotation or purchase consultation,vrf,NaN,NaN,/Bhubaneswar/India,ceo/founder,LGEIL,less than 3 months,1,0,0.003079,0.026846,corporate / office,IT/Software,3,True
4,1.00,/Hyderabad/India,AS,0.088889,17126,Specifier/ Influencer,Enterprise,NaN,NaN,NaN,NaN,consulting,97,Quotation or purchase consultation,multi-split,NaN,NaN,/Hyderabad/India,partner,LGEIL,less than 3 months,0,0,0.003079,0.026846,corporate / office,NaN,4,True
5,1.00,/Abuja/Nigeria,AS,0.040816,16328,End-Customer,SMB,NaN,NaN,NaN,NaN,program and project management,1114,Quotation or purchase consultation,chiller,NaN,NaN,/Abuja/Nigeria,manager,LGEAF,less than 3 months,1,0,0.003079,0.026846,corporate / office,Engineering,5,True
6,0.75,"/Jeddah, KSA/Saudi Arabia",AS,0.040816,20664,End-Customer,SMB,NaN,NaN,NaN,NaN,engineering,420,Quotation or purchase consultation,single-split,NaN,NaN,"/Jeddah, KSA/Saudi Arabia",manager,LGESJ,NaN,1,0,0.003079,0.026846,corporate / office,Engineering,6,True
7,1.00,/Guwahati/India,AS,0.088889,17983,End-Customer,SMB,NaN,NaN,NaN,NaN,sales,205,Quotation or purchase consultation,vrf,NaN,NaN,/Guwahati/India,partner,LGEIL,less than 3 months,1,0,0.003079,0.026846,corporate / office,Manufacturing,7,True
8,0.75,/Cebu city/Philippines,AS,0.066667,30867,Specifier/ Influencer,Enterprise,3.0,NaN,NaN,NaN,other,103,Quotation or purchase consultation,multi-split,NaN,NaN,/Cebu city/Philippines,vice president,LGEPH,less than 3 months,0,0,0.003079,0.026846,corporate / office,Construction,8,True
9,0.75,"/hauz khas,delhi/India",AS,0.088889,6084,End-Customer,SMB,NaN,NaN,NaN,NaN,other,252,Quotation or purchase consultation,vrf,NaN,NaN,"/hauz khas,delhi/India",manager,LGEIL,3 months ~ 6 months,1,0,0.003079,0.026846,corporate / office,NaN,9,True


In [6]:
df_train['business_unit'].unique()

array(['AS', 'ID', 'IT', 'Solution', 'CM'], dtype=object)

In [7]:
# 결측치가 있는 변수 및 결측률 파악 

mis_val = df_train.isnull().sum()
mis_val_bool = mis_val >= 1
mis_val_df = pd.concat([mis_val, mis_val_bool], axis = 1)
mis_val_df = mis_val_df.rename(columns = {0 : 'mis_val', 1 : 'mis_val_bool'})

mis_val_data = mis_val_df.loc[mis_val_df['mis_val_bool'] == True, :]
mis_val_data['ratio'] = mis_val_data['mis_val'] / 59299
mis_val_data_sorted = mis_val_data.sort_values(by='ratio', ascending=True)
mis_val_data_sorted

/tmp/ipykernel_383/929980573.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mis_val_data['ratio'] = mis_val_data['mis_val'] / 59299


,mis_val,mis_val_bool,ratio
inquiry_type,941,True,0.015869
customer_country.1,982,True,0.016560
customer_country,982,True,0.016560
customer_job,18733,True,0.315908
product_category,19374,True,0.326717
expected_timeline,30863,True,0.520464
business_area,40882,True,0.689421
ver_win_rate_x,40882,True,0.689421
customer_type,43961,True,0.741345
ver_win_ratio_per_bu,43995,True,0.741918


In [8]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59299 entries, 0 to 59298
Data columns (total 29 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   bant_submit              59299 non-null  float64
 1   customer_country         58317 non-null  object 
 2   business_unit            59299 non-null  object 
 3   com_reg_ver_win_rate     14568 non-null  float64
 4   customer_idx             59299 non-null  int64  
 5   customer_type            15338 non-null  object 
 6   enterprise               59299 non-null  object 
 7   historical_existing_cnt  13756 non-null  float64
 8   id_strategic_ver         3444 non-null   float64
 9   it_strategic_ver         1121 non-null   float64
 10  idit_strategic_ver       4565 non-null   float64
 11  customer_job             40566 non-null  object 
 12  lead_desc_length         59299 non-null  int64  
 13  inquiry_type             58358 non-null  object 
 14  product_category      

### 전처리
customer_country.1 열 삭제

float -> 범주형 데이터로 변환 ['it_strategic_ver','id_strategic_ver','idit_strategic_ver','lead_owner','ver_cus','ver_pro','customer_idx']

inquiry_type,customer_country - 대소문자 통합

customer_country 결측치 행 삭제

이상치 - MinMaxScale

할거 - 나라,지역 전처리. 다르게 표시된 str 처리

컬럼 중요도 처리

In [9]:
# customer_country , customer_country.1의 값이 일치함.
pre_train = df_train.drop(labels = ['customer_country.1'],axis=1)
pre_test = df_test.drop(labels = ['customer_country.1','id'],axis=1)

dtypes = pre_train.dtypes

# 각 열의 고유값 개수 확인
unique_value_counts = pre_train.nunique()

# 수치형 변수가 아닌 컬럼을 저장할 리스트 초기화
object_columns = []

# 각 열에 대해 데이터 타입이 'object'이거나 고유값 개수가 일정 이하인 경우를 확인하여 non_numeric_columns 리스트에 추가
for col in pre_train.columns:
    if dtypes[col] == 'object':
        object_columns.append(col)

# 수치형 변수가 아닌 컬럼 출력
print("범주형 컬럼:", object_columns,"\n")
print("수치형 컬럼 :", list(set(pre_train.columns)-set(object_columns)))

범주형 컬럼: ['customer_country', 'business_unit', 'customer_type', 'enterprise', 'customer_job', 'inquiry_type', 'product_category', 'product_subcategory', 'product_modelname', 'customer_position', 'response_corporate', 'expected_timeline', 'business_area', 'business_subarea'] 

수치형 컬럼 : ['historical_existing_cnt', 'idit_strategic_ver', 'bant_submit', 'ver_win_rate_x', 'com_reg_ver_win_rate', 'ver_win_ratio_per_bu', 'customer_idx', 'lead_owner', 'is_converted', 'id_strategic_ver', 'lead_desc_length', 'ver_pro', 'it_strategic_ver', 'ver_cus']


In [10]:
"""
범주형 데이터에 넣을 것들 후보
'it_strategic_ver' 'id_strategic_ver' 'idit_strategic_ver' -> array([nan,  1.])
"""
columns_categorized = ['it_strategic_ver','id_strategic_ver','idit_strategic_ver'] + ['lead_owner','ver_cus','ver_pro','customer_idx']

for n in columns_categorized:
    pre_train[n] = pre_train[n].fillna(0)
    pre_train[n] = pre_train[n].astype('int').astype('object')
    print(n,': ',pre_train[n].unique())

it_strategic_ver :  [0 1]
id_strategic_ver :  [0 1]
idit_strategic_ver :  [0 1]
lead_owner :  [0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27
 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51
 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75
 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99
 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 117 119
 120 116 124 125 126 128 130 134 133 140 145 146 147 148 149 150 151 152
 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170
 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188
 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206
 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224
 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242
 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260
 261 262 263 264 265 266 267

In [11]:
object_columns

['customer_country',
 'business_unit',
 'customer_type',
 'enterprise',
 'customer_job',
 'inquiry_type',
 'product_category',
 'product_subcategory',
 'product_modelname',
 'customer_position',
 'response_corporate',
 'expected_timeline',
 'business_area',
 'business_subarea']

In [12]:
pre_train['it_strategic_ver'].head()

0    0
1    0
2    0
3    0
4    0
Name: it_strategic_ver, dtype: object

In [13]:
def preprocess_1(col):
    pre_train[col] = pre_train[col].str.lower()
    pre_test[col] = pre_test[col].str.lower()
    
    mask_train = pre_train[col].notnull()
    mask_test = pre_test[col].notnull()
    pre_train.loc[mask_train, col] = pre_train.loc[mask_train, col].apply(lambda x: x.replace(" ", ""))
    pre_test.loc[mask_test, col] = pre_test.loc[mask_test, col].apply(lambda x: x.replace(" ", ""))    

In [14]:
for n in object_columns:
    preprocess_1(n)
    
#test data에 customer_country = nan 이 없어서 행삭제 해도 될듯. 삭제양도 많지 않음.
pre_train = pre_train.dropna(subset=['customer_country'])

object_columns = object_columns + ['it_strategic_ver','id_strategic_ver','idit_strategic_ver','lead_owner','ver_cus','ver_pro','customer_idx']
numerical_columns = list(set(pre_train.columns)-set(object_columns))

# 보류
# """
pre_train['customer_country'] = pre_train['customer_country'].apply(lambda x: x.split('/')[-1])
pre_test['customer_country'] = pre_test['customer_country'].apply(lambda x: x.split('/')[-1])
# """

In [15]:
pre_train[pre_train['customer_country']==''].count()

bant_submit                1998
customer_country           1998
business_unit              1998
com_reg_ver_win_rate        117
customer_idx               1998
customer_type              1070
enterprise                 1998
historical_existing_cnt     355
id_strategic_ver           1998
it_strategic_ver           1998
idit_strategic_ver         1998
customer_job                828
lead_desc_length           1998
inquiry_type               1998
product_category             50
product_subcategory          36
product_modelname            44
customer_position          1998
response_corporate         1998
expected_timeline          1876
ver_cus                    1998
ver_pro                    1998
ver_win_rate_x              132
ver_win_ratio_per_bu        126
business_area               132
business_subarea             24
lead_owner                 1998
is_converted               1998
dtype: int64

In [16]:
# 각 컬럼별로 이상치 판별 임계값을 계산하는 함수
def find_outlier_thresholds(data, threshold=3):
    thresholds = {}
    min_outliers = {}  # 각 컬럼별 최소 이상치 값을 저장하기 위한 딕셔너리
    
#     num_columns = ['com_reg_ver_win_rate', 'ver_pro', 'ver_win_ratio_per_bu', 
#                    'idit_strategic_ver', 'lead_owner', 'customer_idx', 'bant_submit', 
#                    'id_strategic_ver', 'is_converted', 'historical_existing_cnt', 'it_strategic_ver', 
#                    'ver_win_rate_x', 'lead_desc_length', 'ver_cus']
    
    for column in numerical_columns:
        # 각 컬럼의 Z-Score 계산
        z_scores = (data[column] - data[column].mean()) / data[column].std()
        
        # Z-Score가 임계값보다 큰 데이터를 이상치로 판단
        outliers = data[np.abs(z_scores) > threshold]
        
        # 이상치 판별 임계값 저장
        thresholds[column] = {
            'threshold': threshold,
            'outliers_count': len(outliers),
            'outliers_indices': list(outliers.index)
        }
        
        # 최소 이상치 값 찾기
        if len(outliers) > 0:
            min_outliers[column] = outliers[column].min()
    
    return thresholds, min_outliers

# 예시 데이터 프레임 생성 (스트링 데이터를 가진 'C' 컬럼 추가)

# 이상치 판별 임계값 계산 (임계값은 3으로 설정)
outlier_thresholds, min_outliers = find_outlier_thresholds(pre_train, threshold=3)

# Y_LABEL 값에 따라 이상치 개수와 각 컬럼별 최소 이상치 값을 구분하여 출력
for column, threshold_info in outlier_thresholds.items():
    print(f"Column: {column}")
    print(f"Threshold: {threshold_info['threshold']}")
    
    outliers_true = len([idx for idx in threshold_info['outliers_indices'] if pre_train.loc[idx, 'is_converted'] == True])
    print(f"Number of Outliers : {outliers_true}")
    outliers_false = len([idx for idx in threshold_info['outliers_indices'] if pre_train.loc[idx, 'is_converted'] == False])
    print(f"Number of Outliers : {outliers_false}")
    
    
    # 최소 이상치 값 출력
    if column in min_outliers:
        print(f"Min Outlier Value: {min_outliers[column]}")

    print()


Column: historical_existing_cnt
Threshold: 3
Number of Outliers : 22
Number of Outliers : 43
Min Outlier Value: 158.0

Column: ver_win_rate_x
Threshold: 3
Number of Outliers : 0
Number of Outliers : 0

Column: bant_submit
Threshold: 3
Number of Outliers : 0
Number of Outliers : 0

Column: com_reg_ver_win_rate
Threshold: 3
Number of Outliers : 340
Number of Outliers : 493
Min Outlier Value: 0.6153846153846154

Column: ver_win_ratio_per_bu
Threshold: 3
Number of Outliers : 20
Number of Outliers : 82
Min Outlier Value: 0.2272727272727272

Column: is_converted
Threshold: 3
Number of Outliers : 4715
Number of Outliers : 0
Min Outlier Value: True

Column: lead_desc_length
Threshold: 3
Number of Outliers : 189
Number of Outliers : 1186
Min Outlier Value: 475



In [17]:
features = ['com_reg_ver_win_rate','lead_desc_length','ver_win_ratio_per_bu','historical_existing_cnt']

print(pre_train[features].min())
print(pre_train[features].max())

com_reg_ver_win_rate       0.003788
lead_desc_length           1.000000
ver_win_ratio_per_bu       0.011583
historical_existing_cnt    0.000000
dtype: float64
com_reg_ver_win_rate          1.000000
lead_desc_length           1264.000000
ver_win_ratio_per_bu          0.285714
historical_existing_cnt    1394.000000
dtype: float64


In [18]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

pre_train[features] = scaler.fit_transform(pre_train[features]) # fit_transform
pre_test[features] = scaler.transform(pre_test[features]) # transform

In [19]:
print(pre_train[features].min())
print(pre_train[features].max())

com_reg_ver_win_rate       0.0
lead_desc_length           0.0
ver_win_ratio_per_bu       0.0
historical_existing_cnt    0.0
dtype: float64
com_reg_ver_win_rate       1.0
lead_desc_length           1.0
ver_win_ratio_per_bu       1.0
historical_existing_cnt    1.0
dtype: float64


In [20]:
pre_train[features].head()

,com_reg_ver_win_rate,lead_desc_length,ver_win_ratio_per_bu,historical_existing_cnt
0,0.063118,0.048298,0.055676,NaN
1,0.063118,0.075218,0.055676,0.008608
2,0.085425,0.043547,0.055676,0.103300
3,0.085425,0.034046,0.055676,NaN
4,0.085425,0.076010,0.055676,NaN


In [33]:
pre_train.iloc[[1,2]]

,bant_submit,customer_country,business_unit,com_reg_ver_win_rate,customer_idx,customer_type,enterprise,historical_existing_cnt,id_strategic_ver,it_strategic_ver,idit_strategic_ver,customer_job,lead_desc_length,inquiry_type,product_category,product_subcategory,product_modelname,customer_position,response_corporate,expected_timeline,ver_cus,ver_pro,ver_win_rate_x,ver_win_ratio_per_bu,business_area,business_subarea,lead_owner,is_converted
1,1.0,philippines,as,0.063118,23122,end-customer,enterprise,0.008608,0,0,0,mediaandcommunication,0.075218,quotationorpurchaseconsultation,multi-split,NaN,NaN,ceo/founder,lgeph,lessthan3months,1,0,0.003079,0.055676,corporate/office,advertising,1,True
2,1.0,india,as,0.085425,1755,end-customer,enterprise,0.103300,0,0,0,engineering,0.043547,productinformation,single-split,NaN,NaN,partner,lgeil,lessthan3months,1,0,0.003079,0.055676,corporate/office,construction,2,True


## 3. 모델 학습

### 모델 학습

In [43]:
# Stratified K-Fold 교차 검증 설정
n_splits = 5  # 원하는 Fold 수
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# 최적의 하이퍼파라미터 설정
best_params = {'depth': 10, 'iterations': 1500, 'learning_rate': 0.01}

# 교차 검증 수행 및 평가 지표 저장
macro_f1_scores = []
models = []
best_f1_score = 0

resetIndex_train = pre_train.reset_index(drop=True)

train_x = resetIndex_train.drop(columns=['is_converted'])
train_y = resetIndex_train['is_converted'].astype(int)
train_x = train_x.fillna(0)

class_weights = compute_class_weight('balanced', classes=[0, 1], y=train_y)

for train_index, valid_index in skf.split(train_x, train_y):
    print(train_index)
    print(valid_index)
    X_train, X_valid = train_x.iloc[train_index], train_x.iloc[valid_index]
    y_train, y_valid = train_y[train_index], train_y[valid_index]

    # CatBoost 모델 생성 및 학습
    model = CatBoostClassifier(loss_function='Logloss', cat_features=np.where(pre_train.dtypes == np.object)[0], eval_metric='F1', 
                               random_seed=42, verbose=500, **best_params, class_weights = {0:class_weights[0], 1:class_weights[1]})
    
    model.fit(X_train, y_train, eval_set=[(X_valid,y_valid)])

    # 검증 데이터에 대한 예측 및 평가 (Macro F1 Score)
    valid_predictions = model.predict(X_valid)
    
    print(valid_predictions)
    print(y_valid[:5])
    macro_f1 = f1_score(y_valid, valid_predictions, average='macro')
    macro_f1_scores.append(macro_f1)
    models.append(model)

    if macro_f1 > best_f1_score:
        best_f1_score = macro_f1
        best_classification_model = model
        y_val = y_valid
        pred_val = valid_predictions

# Macro F1 Score 출력
print("Macro F1 Scores:", macro_f1_scores)


[    0     1     2 ... 58312 58313 58316]
[    5    11    16 ... 58309 58314 58315]


/tmp/ipykernel_383/366239352.py:28: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  model = CatBoostClassifier(loss_function='Logloss', cat_features=np.where(pre_train.dtypes == np.object)[0], eval_metric='F1',


0:	learn: 0.8749100	test: 0.8652897	best: 0.8652897 (0)	total: 301ms	remaining: 7m 30s
500:	learn: 0.9683237	test: 0.9511664	best: 0.9516801 (496)	total: 1m 32s	remaining: 3m 5s
1000:	learn: 0.9802611	test: 0.9540508	best: 0.9540508 (998)	total: 3m 3s	remaining: 1m 31s
1499:	learn: 0.9856594	test: 0.9542134	best: 0.9547743 (1494)	total: 4m 36s	remaining: 0us

bestTest = 0.9547743374
bestIteration = 1494

Shrink model to first 1495 iterations.
[1 1 1 ... 0 0 0]
5     1
11    1
16    1
17    1
20    0
Name: is_converted, dtype: int64
[    1     2     3 ... 58313 58314 58315]
[    0     9    23 ... 58301 58311 58316]


/tmp/ipykernel_383/366239352.py:28: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  model = CatBoostClassifier(loss_function='Logloss', cat_features=np.where(pre_train.dtypes == np.object)[0], eval_metric='F1',


0:	learn: 0.8750949	test: 0.8812869	best: 0.8812869 (0)	total: 201ms	remaining: 5m 1s
500:	learn: 0.9669545	test: 0.9482205	best: 0.9493064 (457)	total: 1m 32s	remaining: 3m 4s
1000:	learn: 0.9794610	test: 0.9472877	best: 0.9493064 (457)	total: 2m 59s	remaining: 1m 29s
1499:	learn: 0.9836571	test: 0.9463129	best: 0.9493064 (457)	total: 4m 13s	remaining: 0us

bestTest = 0.9493063823
bestIteration = 457

Shrink model to first 458 iterations.
[0 1 0 ... 0 0 0]
0     1
9     1
23    0
25    0
28    0
Name: is_converted, dtype: int64
[    0     2     3 ... 58314 58315 58316]
[    1     4     7 ... 58295 58298 58299]


...

0:	learn: 0.8696189	test: 0.8772613	best: 0.8772613 (0)	total: 164ms	remaining: 4m 6s
500:	learn: 0.9658189	test: 0.9617092	best: 0.9617092 (499)	total: 1m 33s	remaining: 3m 5s
1000:	learn: 0.9780892	test: 0.9584561	best: 0.9617092 (499)	total: 3m 2s	remaining: 1m 30s
1499:	learn: 0.9845072	test: 0.9551917	best: 0.9617092 (499)	total: 4m 26s	remaining: 0us

bestTest = 0.9617091998
bestIteration = 499

Shrink model to first 500 iterations.
[1 1 1 ... 0 0 0]
1     1
4     1
7     1
14    1
15    1
Name: is_converted, dtype: int64
[    0     1     4 ... 58314 58315 58316]
[    2     3     6 ... 58305 58310 58313]


/tmp/ipykernel_383/366239352.py:28: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  model = CatBoostClassifier(loss_function='Logloss', cat_features=np.where(pre_train.dtypes == np.object)[0], eval_metric='F1',


0:	learn: 0.8641126	test: 0.8602069	best: 0.8602069 (0)	total: 213ms	remaining: 5m 18s
500:	learn: 0.9668466	test: 0.9486959	best: 0.9492579 (485)	total: 1m 32s	remaining: 3m 5s
1000:	learn: 0.9783303	test: 0.9478058	best: 0.9501010 (702)	total: 3m 4s	remaining: 1m 31s
1499:	learn: 0.9845816	test: 0.9455754	best: 0.9501010 (702)	total: 4m 34s	remaining: 0us

bestTest = 0.9501009935
bestIteration = 702

Shrink model to first 703 iterations.
[1 1 1 ... 0 0 0]
2     1
3     1
6     1
8     1
22    0
Name: is_converted, dtype: int64
[    0     1     2 ... 58314 58315 58316]
[   10    12    13 ... 58300 58307 58312]


/tmp/ipykernel_383/366239352.py:28: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  model = CatBoostClassifier(loss_function='Logloss', cat_features=np.where(pre_train.dtypes == np.object)[0], eval_metric='F1',


0:	learn: 0.8418136	test: 0.8566825	best: 0.8566825 (0)	total: 51.9ms	remaining: 1m 17s
500:	learn: 0.9668783	test: 0.9579561	best: 0.9579561 (495)	total: 1m 34s	remaining: 3m 7s
1000:	learn: 0.9789804	test: 0.9567597	best: 0.9586570 (636)	total: 2m 57s	remaining: 1m 28s
1499:	learn: 0.9810951	test: 0.9563184	best: 0.9586570 (636)	total: 4m 1s	remaining: 0us

bestTest = 0.95865697
bestIteration = 636

Shrink model to first 637 iterations.
[1 1 1 ... 0 0 0]
10    1
12    1
13    1
18    0
19    0
Name: is_converted, dtype: int64
Macro F1 Scores: [0.89690428411685, 0.8878082579688455, 0.8970321169721218, 0.8846391985494577, 0.889816241749436]


In [44]:
def get_clf_eval(y_test, y_pred=None):
    confusion = confusion_matrix(y_test, y_pred, labels=[1, 0])
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, labels=[True, False])
    recall = recall_score(y_test, y_pred)
    F1 = f1_score(y_test, y_pred, labels=[True, False])

    print("오차행렬:\n", confusion)
    print("\n정확도: {:.4f}".format(accuracy))
    print("정밀도: {:.4f}".format(precision))
    print("재현율: {:.4f}".format(recall))
    print("F1: {:.4f}".format(F1))

In [45]:
get_clf_eval(y_val, pred_val)

오차행렬:
 [[  904    39]
 [  375 10345]]

정확도: 0.9645
정밀도: 0.7068
재현율: 0.9586
F1: 0.8137


## 4. 제출하기

### 테스트 데이터 예측

In [46]:
for n in columns_categorized:
    pre_test[n] = pre_test[n].fillna(0)
    pre_test[n] = pre_test[n].astype('int').astype('object')

In [ ]:
# 예측에 필요한 데이터 분리
x_test = pre_test.drop(["is_converted"], axis=1)
x_test.head()

In [ ]:
train_x.head()

In [ ]:
x_test.info()

In [ ]:
test_pred = best_classification_model.predict(x_test.fillna(0))
print(test_pred)
sum(int(pred == 1) for pred in test_pred)

### 제출 파일 작성

In [ ]:
# 제출 데이터 읽어오기 (df_test는 전처리된 데이터가 저장됨)
df_sub = pd.read_csv("submission.csv")
df_sub["is_converted"] = test_pred

# 제출 파일 저장
df_sub.to_csv("submission.csv", index=False)

In [ ]:
best_classification_model.get_feature_importance(prettified=True)

**우측 상단의 제출 버튼을 클릭해 결과를 확인하세요**